# Session 2 Scraping
- Why is scrapping useful?
- Basics of How Browser Works
- How to clean from json, 
- Wikipeadia getting back to Phylisophy
- Twitter API
- Reading Data from DataBase db

In [78]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [79]:
import re
import bs4
import requests

<p style='font-size:20px'>
Suppose that we wish to build a system that warns us when stock prices starts to go up or go down. The first thing we want to do is to scrape a website to get the data. <br>
Specifically, get data from website https://finance.yahoo.com/most-active
</p>

In [83]:
url = 'https://finance.yahoo.com/most-active'
res = requests.get(url=url)
soup = bs4.BeautifulSoup(res.text, "html.parser")

In [81]:
res

<Response [200]>

In [84]:
with open('page.html','a') as f:
    f.write(soup.prettify())

In [86]:
# print(soup.prettify())
tabl = soup.select_one('#scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table') ## Copied using Chrome Extension #scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table
# #scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table

thead = tabl.find('thead')
tbody = tabl.find('tbody')

head_lst = [hd.text for hd in thead.find_all('th')]

tab_lst = list()
for tr in tbody.find_all('tr'):
    tr_lst = list()
    for td in tr.find_all('td'):
        tr_lst.append(td.text)
    tab_lst.append(tr_lst)

In [88]:
df_stock = pd.DataFrame(tab_lst, columns=head_lst)
df_stock

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
0,AMD,"Advanced Micro Devices, Inc.",83.75,-0.04,-0.05%,86.467M,108.621M,135.718B,31.43,
1,NIO,NIO Inc.,22.55,-0.11,-0.49%,74.775M,71.608M,37.67B,N/A,
2,AAPL,Apple Inc.,135.35,-0.52,-0.38%,73.165M,96.051M,2.191T,22.05,
3,AMZN,"Amazon.com, Inc.",108.95,+0.27,+0.25%,58.016M,87.079M,1.109T,52.25,
4,LU,Lufax Holding Ltd,6.46,-0.36,-5.28%,48.012M,8.579M,14.766B,6.32,
5,T,AT&T Inc.,20.32,+0.36,+1.80%,45.622M,52.356M,145.471B,8.55,
6,VALE,Vale S.A.,14.48,-0.24,-1.63%,45.529M,34.345M,74.278B,3.24,
7,F,Ford Motor Company,11.48,+0.02,+0.17%,45.411M,63.517M,45.333B,4.00,
8,ITUB,Itaú Unibanco Holding S.A.,4.5700,-0.0800,-1.72%,42.271M,48.817M,44.694B,8.31,
9,META,"Meta Platforms, Inc.",155.85,-1.20,-0.76%,46.804M,31.677M,421.78B,11.79,


In [89]:
df_stock.dtypes

Symbol               object
Name                 object
Price (Intraday)     object
Change               object
% Change             object
Volume               object
Avg Vol (3 month)    object
Market Cap           object
PE Ratio (TTM)       object
52 Week Range        object
dtype: object

In [90]:
df_stock['Change'] = df_stock['Change'].astype(float)
df_stock['% Change'] = df_stock['% Change'].str.rstrip('%').astype('float') / 100.0
df_stock['Volume'] = df_stock['Volume'].str.rstrip('M').astype(float)
df_stock['Avg Vol (3 month)'] = df_stock['Avg Vol (3 month)'].str.rstrip('M').astype(float)
df_stock['Price (Intraday)'] = df_stock['Price (Intraday)'].astype(float)

In [91]:
df_stock.dtypes

Symbol                object
Name                  object
Price (Intraday)     float64
Change               float64
% Change             float64
Volume               float64
Avg Vol (3 month)    float64
Market Cap            object
PE Ratio (TTM)        object
52 Week Range         object
dtype: object

In [92]:
df_stock.describe()

,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month)
count,25.000000,25.000000,25.000000,25.000000,25.000000
mean,67.346000,-0.474000,-0.006524,44.728880,43.813200
std,142.267244,1.178866,0.026489,14.232608,25.687321
min,4.570000,-4.200000,-0.091900,32.091000,2.872000
25%,9.010000,-0.360000,-0.011900,34.102000,31.185000
50%,20.320000,-0.080000,-0.004900,42.271000,36.432000
75%,55.770000,0.050000,0.002500,45.622000,52.356000
max,708.260000,1.480000,0.049900,86.467000,108.621000


In [93]:
df_stock[df_stock['Change'] == df_stock['Change'].max()]

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
19,RBLX,Roblox Corporation,31.12,1.48,0.0499,32.73,28.876,18.46B,N/A,


In [94]:
df_stock[df_stock['Change'] == df_stock['Change'].min()]

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
13,MO,"Altria Group, Inc.",41.5,-4.2,-0.0919,41.715,10.891,75.138B,25.40,


## Exercise 1: Scrape the Table from Cancer Statistics
Go to the site https://www.cdc.gov/cancer/kinds.htm
Scrape the explaination for each cancer in the first cards, store it in a dictionary.

In [96]:
url = 'https://www.cdc.gov/cancer/kinds.htm'
res = requests.get(url=url)
soup = bs4.BeautifulSoup(res.text, "html.parser")

In [97]:
# 71413
tabl = soup.select_one('body > div.container.d-flex.flex-wrap.body-wrapper.bg-white > main > div:nth-child(3) > div > div:nth-child(4)') ## Copied using Chrome Extension #scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table
# #scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table

ccernms = tabl.find_all('div', {'class':'card-header'})
ccerepls = tabl.find_all('div',{'class':'card-body'})

In [101]:
{i:j for i,j in zip(range(0,10), range(10,20))}

{0: 10, 1: 11, 2: 12, 3: 13, 4: 14, 5: 15, 6: 16, 7: 17, 8: 18, 9: 19}

In [102]:
dict_i =  {}
for i,j in zip(range(0,10), range(10,20)):
    dict_i[i] = j
dict_i

{0: 10, 1: 11, 2: 12, 3: 13, 4: 14, 5: 15, 6: 16, 7: 17, 8: 18, 9: 19}

In [103]:
res = {elei.text : elej.text for elei, elej in zip(ccernms,ccerepls) }

In [104]:
res

{'Bladder Cancer': 'Bladder cancer risk factors include smoking, genetic mutations, and exposure to certain chemicals.\n',
 'Breast Cancer': 'Bladder cancer risk factors include smoking, genetic mutations, and exposure to certain chemicals.\n',
 'Cervical Cancer': 'Regular mammograms are the best tests doctors have to find breast cancer early. Talk to your doctor about when to start and how often to get a screening mammogram.\n',
 'Colorectal Cancer': 'Regular mammograms are the best tests doctors have to find breast cancer early. Talk to your doctor about when to start and how often to get a screening mammogram.\n',
 'Gynecologic Cancers': 'Screening tests and the HPV vaccine can help prevent cervical cancer.\n',
 'Head and Neck Cancers': 'Screening tests and the HPV vaccine can help prevent cervical cancer.\n',
 'Kidney Cancer': 'If you are 45 years old or older, get screened. Screening tests can help prevent colorectal cancer or find it early, when treatment works best.\n',
 'Liver 

## Dynamic Loading and Writing CSV files

In [105]:
df_stock

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
0,AMD,"Advanced Micro Devices, Inc.",83.75,-0.04,-0.0005,86.467,108.621,135.718B,31.43,
1,NIO,NIO Inc.,22.55,-0.11,-0.0049,74.775,71.608,37.67B,N/A,
2,AAPL,Apple Inc.,135.35,-0.52,-0.0038,73.165,96.051,2.191T,22.05,
3,AMZN,"Amazon.com, Inc.",108.95,0.27,0.0025,58.016,87.079,1.109T,52.25,
4,LU,Lufax Holding Ltd,6.46,-0.36,-0.0528,48.012,8.579,14.766B,6.32,
5,T,AT&T Inc.,20.32,0.36,0.0180,45.622,52.356,145.471B,8.55,
6,VALE,Vale S.A.,14.48,-0.24,-0.0163,45.529,34.345,74.278B,3.24,
7,F,Ford Motor Company,11.48,0.02,0.0017,45.411,63.517,45.333B,4.00,
8,ITUB,Itaú Unibanco Holding S.A.,4.57,-0.08,-0.0172,42.271,48.817,44.694B,8.31,
9,META,"Meta Platforms, Inc.",155.85,-1.20,-0.0076,46.804,31.677,421.78B,11.79,


In [106]:
df_stock.to_csv('stocks.csv', index=False)

In [107]:
import csv

In [108]:
f = open('stocks.csv')

In [110]:
reader = csv.reader(f)

In [111]:
header = next(reader)

In [113]:
rows = []
for rw in reader:
    rows.append(rw)

In [114]:
pd.DataFrame(rows, columns=header)

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
0,AMD,"Advanced Micro Devices, Inc.",83.75,-0.04,-0.0005,86.467,108.621,135.718B,31.43,
1,NIO,NIO Inc.,22.55,-0.11,-0.0049,74.775,71.608,37.67B,N/A,
2,AAPL,Apple Inc.,135.35,-0.52,-0.0038,73.165,96.051,2.191T,22.05,
3,AMZN,"Amazon.com, Inc.",108.95,0.27,0.0025,58.016,87.079,1.109T,52.25,
4,LU,Lufax Holding Ltd,6.46,-0.36,-0.0528,48.012,8.579,14.766B,6.32,
5,T,AT&T Inc.,20.32,0.36,0.018000000000000002,45.622,52.356,145.471B,8.55,
6,VALE,Vale S.A.,14.48,-0.24,-0.0163,45.529,34.345,74.278B,3.24,
7,F,Ford Motor Company,11.48,0.02,0.0017000000000000001,45.411,63.517,45.333B,4.00,
8,ITUB,Itaú Unibanco Holding S.A.,4.57,-0.08,-0.0172,42.271,48.817,44.694B,8.31,
9,META,"Meta Platforms, Inc.",155.85,-1.2,-0.0076,46.804,31.677,421.78B,11.79,


In [115]:
f.close()

In [47]:
rows = []

In [116]:
with open('stocks.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    rows = []
    for rw in reader:
        rows.append(rw)

In [118]:
pd.DataFrame(rows, columns=header)

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
0,AMD,"Advanced Micro Devices, Inc.",83.75,-0.04,-0.0005,86.467,108.621,135.718B,31.43,
1,NIO,NIO Inc.,22.55,-0.11,-0.0049,74.775,71.608,37.67B,N/A,
2,AAPL,Apple Inc.,135.35,-0.52,-0.0038,73.165,96.051,2.191T,22.05,
3,AMZN,"Amazon.com, Inc.",108.95,0.27,0.0025,58.016,87.079,1.109T,52.25,
4,LU,Lufax Holding Ltd,6.46,-0.36,-0.0528,48.012,8.579,14.766B,6.32,
5,T,AT&T Inc.,20.32,0.36,0.018000000000000002,45.622,52.356,145.471B,8.55,
6,VALE,Vale S.A.,14.48,-0.24,-0.0163,45.529,34.345,74.278B,3.24,
7,F,Ford Motor Company,11.48,0.02,0.0017000000000000001,45.411,63.517,45.333B,4.00,
8,ITUB,Itaú Unibanco Holding S.A.,4.57,-0.08,-0.0172,42.271,48.817,44.694B,8.31,
9,META,"Meta Platforms, Inc.",155.85,-1.2,-0.0076,46.804,31.677,421.78B,11.79,


In [119]:
def scrape_yahoo():
    url = 'https://finance.yahoo.com/most-active'
    res = requests.get(url=url)
    soup = bs4.BeautifulSoup(res.text, "html.parser")

    tabl = soup.select_one('#scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table') ## Copied using Chrome Extension #scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table
    # #scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table
    
    thead = tabl.find('thead')
    tbody = tabl.find('tbody')
    
    head_lst = [hd.text for hd in thead.find_all('th')]
    
    tab_lst = list()
    for tr in tbody.find_all('tr'):
        tr_lst = list()
        for td in tr.find_all('td'):
            tr_lst.append(td.text)
        tab_lst.append(tr_lst)

    return pd.DataFrame(tab_lst, columns=head_lst)

In [120]:
df_stock = scrape_yahoo()

In [121]:
stockdic = df_stock.transpose().to_dict()

In [124]:
stockdic

{0: {'Symbol': 'AMD',
  'Name': 'Advanced Micro Devices, Inc.',
  'Price (Intraday)': '83.75',
  'Change': '-0.04',
  '% Change': '-0.05%',
  'Volume': '86.467M',
  'Avg Vol (3 month)': '108.621M',
  'Market Cap': '135.718B',
  'PE Ratio (TTM)': '31.43',
  '52 Week Range': ''},
 1: {'Symbol': 'NIO',
  'Name': 'NIO Inc.',
  'Price (Intraday)': '22.55',
  'Change': '-0.11',
  '% Change': '-0.49%',
  'Volume': '74.775M',
  'Avg Vol (3 month)': '71.608M',
  'Market Cap': '37.67B',
  'PE Ratio (TTM)': 'N/A',
  '52 Week Range': ''},
 2: {'Symbol': 'AAPL',
  'Name': 'Apple Inc.',
  'Price (Intraday)': '135.35',
  'Change': '-0.52',
  '% Change': '-0.38%',
  'Volume': '73.165M',
  'Avg Vol (3 month)': '96.051M',
  'Market Cap': '2.191T',
  'PE Ratio (TTM)': '22.05',
  '52 Week Range': ''},
 3: {'Symbol': 'AMZN',
  'Name': 'Amazon.com, Inc.',
  'Price (Intraday)': '108.95',
  'Change': '+0.27',
  '% Change': '+0.25%',
  'Volume': '58.016M',
  'Avg Vol (3 month)': '87.079M',
  'Market Cap': '1.1

In [122]:
header = df_stock.columns

In [127]:
with open('stocks_appd.csv','a+') as fa:
    csvwriter = csv.DictWriter(fa, fieldnames=header)
    csvwriter.writeheader()
    for key, dic_i in stockdic.items():
        csvwriter.writerow(dic_i)

## Wikipedia Game

### 3. Snscrape
Snscrape provides many useful attributes available through snscrape Twitter object [(Beck, 2020)](https://betterprogramming.pub/how-to-scrape-tweets-with-snscrape-90124ed006af). You might need some of the above attributes when scraping data for your group projects.
* url: Permalink pointing to tweet location
* date: The date on which the tweet was created
* content: The text content of the tweet
* id: The id of the tweet
* user: User object containing the following data: username, displayname, id, description, descriptionUrls, verified, created, followersCount, friendsCount, statusesCount, favouritesCount, listedCount, location, protected, linkUrl, profileImageUrl, profileBannerUrl
* replyCount: The count of replies
* retweetCount: The count of retweets
* likeCount: The count of likes
* quoteCount: The count of users that quoted the tweet and replied
* conversationId: Appears to be the same as tweet id
* lang: Machine generated, assumed language of the tweet
* source: Where tweet was posted from (e.g., iPhone, Andriod, etc.)
* retweetedTweet: The id of the original tweet (if it is a retweet)
* mentionedUsers: The user objects of any mentioned user in the tweet

In [128]:
import snscrape.modules.twitter as sntwitter
from tqdm import tqdm

In [129]:
tweets_list2 = []

# use TwitterSearchScraper to scrape data and append tweets to list
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('let\'s go duke since:2021-11-12 until:2021-11-13').get_items()):
    tweets_list2.append([tweet.date, tweet.content, tweet.user.location])

In [130]:
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Date', 'Content', 'User\'s location'])
tweets_df2

,Date,Content,User's location
0,2021-11-12 23:59:21+00:00,Let's go Duke 🤘,My Own Personal Hell
1,2021-11-12 23:58:46+00:00,Roach\nKeels\nMoore\nBanchero\nWilliams\n\nGra...,
2,2021-11-12 23:57:29+00:00,👏🏻 Let’s 👏🏻 Go ✊🏻 Duke! 🔵😈 https://t.co/H589Xp...,"Germantown, MD"
3,2021-11-12 23:56:40+00:00,Let’s go Duke!! #BeatArmy,#SI6HTS
4,2021-11-12 23:55:37+00:00,LET’S GO DUKE!!! One last ride against where t...,"Raleigh, NC"
5,2021-11-12 23:54:42+00:00,Let’s Go Duke👿 https://t.co/beTdrGHzXW,
6,2021-11-12 23:53:25+00:00,Let’s go Duke!!! 💙😈🏀,
7,2021-11-12 23:51:58+00:00,LET’S GO DUKE 😈,252
8,2021-11-12 23:51:13+00:00,Gametime baby let's go Duke https://t.co/lmXSV...,"San Antonio, TX"
9,2021-11-12 23:48:02+00:00,It’s almost game time Let’s Go Duke #DukeNation,"Atlanta, GA"


### Exercise 3: Scrape twitter related to cancer